In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas
import xarray
import cftime
import tensorflow as tf

/Users/paulag/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
datapath = 'nino34_monthly.nc'
nino34 = xarray.open_dataset(datapath, decode_times = False)
print(nino34)
nino34 = np.array(nino34['nino34'].values)

<xarray.Dataset>
Dimensions:         (bounds: 2, month: 12, time: 7800)
Coordinates:
  * time            (time) float64 15.5 45.0 74.5 ... 2.372e+05 2.372e+05
Dimensions without coordinates: bounds, month
Data variables:
    nino34          (time) float64 ...
    time_bnds       (time, bounds) float64 ...
    areacello       float32 ...
    days_per_month  (month) int32 ...


In [4]:
def ONI(nino34, m = 3):
    oni = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        oni[i] = np.mean(nino34[max(0, (i - m + 1)) : min((i + 1), length)])
    return oni

In [5]:
oni = ONI(nino34)

In [6]:
def climatology(nino34):
    clm = np.zeros(12)
    length = nino34.shape[0]
    for month in range(12):
        section = [12 * i + month for i in range(length // 12)]
        clm[month] = np.mean(nino34[section])
    return clm

In [7]:
clm = climatology(nino34)

In [8]:
def SST_anomaly(nino34, clm):
    anm = np.array(nino34)
    length = nino34.shape[0]
    for i in range(length):
        anm[i] = nino34[i] - clm[i % 12]
    return anm

In [10]:
anm = SST_anomaly(nino34, clm)
oanm = ONI(anm)

# Data Preparation

In [41]:
T = 6                       # prediction timeline
H = 48                      # history used for prediction
include_month = 0           # 1 if we use the month as a feature, 0 otherwise
n_classes = 3               # number of classes (El Nino, El Nina, No Event)
threshold = 0.5         
signal = np.array(nino34)   # data used for training/testing
length = signal.shape[0]    # number of data points
size = length - H - T       # effective dataset size

In [42]:
# create the 'history matrix'
data = np.ndarray((size, H + include_month))
for i in range(size):
    if(include_month == False):
        data[i] = signal[i:(i + H)]
    else:
        data[i] = np.append(signal[i:(i + H)], (i + H + T) % 12)

# label El Nino as 2, El Nina as 0 and no event as 1
labels = np.ndarray((size))
for i in range(length - H - T):
    if(oanm[i + H + T] >= threshold):
        labels[i] = 2
    elif(oanm[i + H + T] <= -threshold):
        labels[i] = 0
    else:
        labels[i] = 1

We split the data into 80% training, 10% validation and 10% testing.

In [43]:
np.random.seed(0)

split = size // 10      
shuffle = np.random.permutation(size)
train_ind = np.array(shuffle[0: 8 * split])
val_ind = np.array(shuffle[(8 * split + 1): 9 * split])
test_ind = np.array(shuffle[(9 * split + 1): size])

train = np.array(data[train_ind])
train_labels = np.array(labels[train_ind])

val = np.array(data[val_ind])
val_labels = np.array(labels[val_ind])

test = np.array(data[test_ind])
test_labels = np.array(labels[test_ind])

# Normalization and Label One-Hot Encoding

In [54]:
mean = np.mean(train)
std = np.std(train)
train_n = (train - mean) / std
val_n = (val - mean) / std
test_n = (test - mean) / std

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    encoded_train_labels = tf.one_hot(train_labels, depth = n_classes).eval()
    encoded_val_labels = tf.one_hot(val_labels, depth = n_classes).eval()
    encoded_test_labels = tf.one_hot(test_labels, depth = n_classes).eval()

# A One Hidden Layer Neural Network

First we try to get an idea about what the simplest neural network architecture can achieve.

In [62]:
for E in [1, 5, 10, 20]:
    for i in [1, 2, 4, 6]:
        for d in [0.0, 0.2, 0.5]:
        
            N = i * H
        
            model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N, activation=tf.nn.relu),
              tf.keras.layers.Dropout(d),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])
            
            model.reset_states()

            model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

            model.fit(train_n, encoded_train_labels, epochs = E, verbose = 0)
            (loss, acc) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
            print(E, " epochs, ", N, " nodes, ", d, " dropout", acc, "% accuracy")

1  epochs,  48  nodes,  0.0  dropout 0.5549805952383043 % accuracy
1  epochs,  48  nodes,  0.2  dropout 0.548512289934294 % accuracy
1  epochs,  48  nodes,  0.5  dropout 0.5446313066362255 % accuracy
1  epochs,  96  nodes,  0.0  dropout 0.5692108667914648 % accuracy
1  epochs,  96  nodes,  0.2  dropout 0.5782664942170793 % accuracy
1  epochs,  96  nodes,  0.5  dropout 0.5536869341775024 % accuracy
1  epochs,  192  nodes,  0.0  dropout 0.5705045279679293 % accuracy
1  epochs,  192  nodes,  0.2  dropout 0.5679172057692169 % accuracy
1  epochs,  192  nodes,  0.5  dropout 0.5730918499738711 % accuracy
1  epochs,  288  nodes,  0.0  dropout 0.5523932731167003 % accuracy
1  epochs,  288  nodes,  0.2  dropout 0.5782664943327417 % accuracy
1  epochs,  288  nodes,  0.5  dropout 0.5834411384602876 % accuracy
5  epochs,  48  nodes,  0.0  dropout 0.5485122898186318 % accuracy
5  epochs,  48  nodes,  0.2  dropout 0.5730918499738711 % accuracy
5  epochs,  48  nodes,  0.5  dropout 0.5601552393658503 %

The results above are consistent with the intuition that more epochs and more nodes produce better results. They also showcase that 0.5 dropout might be too much. Although from the results above we may think that dropout is not that useful, it usually is the case that deeper networks benefit more from the addition of dropout layers and therefore we will not discount the technique.

# Optimizing Deep Neural Network Architecture

We will fix the number of epochs to 25 and vary dropout, the number of layers and the number of nodes.

In [71]:
epochs = 25
loss = {}
acc = {}

In [74]:
for l in [2, 3, 4]:
    for i in [4, 8, 16]:
        for d in [0.2, 0.3, 0.5]:
            
            N = i * H
            
            layers = []
            
            for n in range(l): 
                layers.append(tf.keras.layers.Dense(N, activation=tf.nn.relu))
                layers.append(tf.keras.layers.Dropout(d))
            
            layers.append(tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax))
            
            model = tf.keras.models.Sequential(layers)

            model.reset_states()

            model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                    metrics=['accuracy'])

            model.fit(train_n, encoded_train_labels, epochs = epochs, verbose = 0)
            (loss[(l, i, d)], acc[(l, i, d)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
            print(l, " layers, ", N, " nodes, ", d, " dropout", acc[(l, i, d)], "% accuracy")

2  layers,  192  nodes,  0.2  dropout 0.645536869378787 % accuracy
2  layers,  192  nodes,  0.3  dropout 0.6222509702843497 % accuracy
2  layers,  192  nodes,  0.5  dropout 0.6481241916160534 % accuracy
2  layers,  384  nodes,  0.2  dropout 0.676584734953699 % accuracy
2  layers,  384  nodes,  0.3  dropout 0.6778783960145011 % accuracy
2  layers,  384  nodes,  0.5  dropout 0.6804657181361051 % accuracy
2  layers,  768  nodes,  0.2  dropout 0.6545924968044015 % accuracy
2  layers,  768  nodes,  0.3  dropout 0.7128072446561569 % accuracy
2  layers,  768  nodes,  0.5  dropout 0.684346701202849 % accuracy
3  layers,  192  nodes,  0.2  dropout 0.6895213455617196 % accuracy
3  layers,  192  nodes,  0.3  dropout 0.66364812426857 % accuracy
3  layers,  192  nodes,  0.5  dropout 0.6662354463516201 % accuracy
3  layers,  384  nodes,  0.2  dropout 0.714100905716959 % accuracy
3  layers,  384  nodes,  0.3  dropout 0.7192755499601673 % accuracy
3  layers,  384  nodes,  0.5  dropout 0.69728331184942

# Testing whether more epochs are necessary:

We will pick the top 5 best architectures from above and check if any of them would benefit from more epochs.

In [75]:
top5 = [(2, 768, 0.3), (3, 384, 0.2), (3, 384, 0.3), (3, 768, 0.5), (4, 768, 0.5)]
acc_e = {}
loss_e = {}

In [76]:
for e in [30, 40, 50]:
    for m in top5:
        
        layers = []
        
        (l, N, d) = m
        
        for n in range(l): 
            layers.append(tf.keras.layers.Dense(N, activation=tf.nn.relu))
            layers.append(tf.keras.layers.Dropout(d))
            
        layers.append(tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax))
            
        model = tf.keras.models.Sequential(layers)

        model.reset_states()

        model.compile(optimizer='adam',
            loss='categorical_crossentropy',
                 metrics=['accuracy'])

        model.fit(train_n, encoded_train_labels, epochs = e, verbose = 0)
        (loss_e[(e, l, i, d)], acc_e[(e, l, i, d)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
        print(e, "epochs", l, " layers, ", N, " nodes, ", d, " dropout", acc_e[(e, l, i, d)], "% accuracy")

30 epochs 2  layers,  768  nodes,  0.3  dropout 0.6714100907104906 % accuracy
30 epochs 3  layers,  384  nodes,  0.2  dropout 0.7347994826126839 % accuracy
30 epochs 3  layers,  384  nodes,  0.3  dropout 0.7128072445790486 % accuracy
30 epochs 3  layers,  768  nodes,  0.5  dropout 0.7089262614737507 % accuracy
30 epochs 4  layers,  768  nodes,  0.5  dropout 0.7115135835953549 % accuracy
40 epochs 2  layers,  768  nodes,  0.3  dropout 0.7102199225345528 % accuracy
40 epochs 3  layers,  384  nodes,  0.2  dropout 0.7153945666620988 % accuracy
40 epochs 3  layers,  384  nodes,  0.3  dropout 0.73350582162899 % accuracy
40 epochs 3  layers,  768  nodes,  0.5  dropout 0.6908150065454135 % accuracy
40 epochs 4  layers,  768  nodes,  0.5  dropout 0.6998706340481362 % accuracy
50 epochs 2  layers,  768  nodes,  0.3  dropout 0.7322121605681879 % accuracy
50 epochs 3  layers,  384  nodes,  0.2  dropout 0.7542043986018231 % accuracy
50 epochs 3  layers,  384  nodes,  0.3  dropout 0.7063389391979301

# Varying individual layer widths 

Based on the results above, it makes sense to stick to a 3 hidden layer architecture and train it for 50 epochs. Sometimes it helps if the network has layers of different widths. We will test if this is the case in our problem below. We will still vary dropout because it is still unclear whether we should go for a conservative rate of ~0.2 or for the harsher 0.5.

In [80]:
epochs = 50
loss_w = {}
acc_w = {}

In [82]:
for i1 in [4, 8, 16, 32]:
    for i2 in [4, 8, 16, 32]:
        for i3 in [4, 8, 16, 32]:
            for d in [0.2, 0.3, 0.5]:
            
                model = tf.keras.models.Sequential([
                  tf.keras.layers.Dense(i1 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(i2 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(i3 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
                ])

                model.reset_states()

                model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

                model.fit(train_n, encoded_train_labels, epochs = epochs, verbose = 0)
                (loss_w[(d, i1, i2, i3)], acc_w[(d, i1, i2, i3)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
                print("dropout:", d, "layers of widths", i1 * H, i2 * H, i3 * H, "=>", acc_w[(d, i1, i2, i3)], "% accuracy")

dropout: 0.2 layers of widths 192 192 192 => 0.7050452782142362 % accuracy
dropout: 0.3 layers of widths 192 192 192 => 0.7011642949932759 % accuracy
dropout: 0.5 layers of widths 192 192 192 => 0.7218628720817714 % accuracy
dropout: 0.2 layers of widths 192 192 384 => 0.7166882277614549 % accuracy
dropout: 0.3 layers of widths 192 192 384 => 0.6804657180589969 % accuracy
dropout: 0.5 layers of widths 192 192 384 => 0.6921086676833238 % accuracy
dropout: 0.2 layers of widths 192 192 768 => 0.7192755499601673 % accuracy
dropout: 0.3 layers of widths 192 192 768 => 0.7011642951089383 % accuracy
dropout: 0.5 layers of widths 192 192 768 => 0.7231565331425734 % accuracy
dropout: 0.2 layers of widths 192 192 1536 => 0.7231565330269112 % accuracy
dropout: 0.3 layers of widths 192 192 1536 => 0.6921086676833238 % accuracy
dropout: 0.5 layers of widths 192 192 1536 => 0.7153945666620988 % accuracy
dropout: 0.2 layers of widths 192 384 192 => 0.717981888822257 % accuracy
dropout: 0.3 layers of 

KeyboardInterrupt: 

In [ ]:
for i1 in [4, 8, 16, 32]:
    for i2 in [8, 16, 32]:
        for i3 in [4, 8, 16, 32]:
            for d in [0.2, 0.3, 0.5]:
                
                if(i3 == 4 and d != 0.5):
                    break
            
                model = tf.keras.models.Sequential([
                  tf.keras.layers.Dense(i1 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(i2 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(i3 * H, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
                ])

                model.reset_states()

                model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

                model.fit(train_n, encoded_train_labels, epochs = epochs, verbose = 0)
                (loss_w[(d, i1, i2, i3)], acc_w[(d, i1, i2, i3)]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
                print("dropout:", d, "layers of widths", i1 * H, i2 * H, i3 * H, "=>", acc_w[(d, i1, i2, i3)], "% accuracy")

dropout: 0.2 layers of widths 192 384 384 => 0.6985769729873341 % accuracy
dropout: 0.3 layers of widths 192 384 384 => 0.7244501940877133 % accuracy
dropout: 0.5 layers of widths 192 384 384 => 0.7076326004129486 % accuracy
dropout: 0.2 layers of widths 192 384 768 => 0.7205692109438612 % accuracy
dropout: 0.3 layers of widths 192 384 768 => 0.7360931436349318 % accuracy
dropout: 0.5 layers of widths 192 384 768 => 0.7102199223996134 % accuracy
dropout: 0.2 layers of widths 192 384 1536 => 0.7179818888993652 % accuracy


Let's inspect which architectures did better than 72% accuracy:

In [ ]:
{k:v for (k,v) in acc_w.items() if v > 0.72}

In [ ]:
(N1, N2, N3) = (4 * H, 8 * H, 8 * H)

# Introducing Batch Normalization

Batch normalization may synergize with dropout so we vary d when inspecting the effectiveness of batch normalization:

In [ ]:
for d in [0.2, 0.3, 0.5]:
    model = tf.keras.models.Sequential([
                  tf.keras.layers.Dense(N1, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.BatchNormalization(),
                  tf.keras.layers.Dense(N2, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.BatchNormalization(),
                  tf.keras.layers.Dense(N3, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(d),
                  tf.keras.layers.BatchNormalization(),
                  tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
                ])

    model.reset_states()

    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

    model.fit(train_n, encoded_train_labels, epochs = epochs)
    print("Dropout: ", d)
    model.evaluate(val_n, encoded_val_labels)

# Exploring various activation functions

In [ ]:
acc_f = np.zeros(6)
loss_f = np.zeros(6)
function_name = ["sigmoid", "hard sigmoid", "tanh", "relu", "leaky relu", "selu"]
t = 0

In [ ]:
def leaky_relu_01(x):
    return tf.nn.leaky_relu(x, alpha=0.01)

In [ ]:
for f in [tf.nn.sigmoid, tf.nn.hard_sigmoid, tf.nn.tanh, tf.nn.relu, leaky_relu_01, tf.nn.selu]:
    
    model = tf.keras.models.Sequential([
              tf.keras.layers.Dense(N1, activation=f),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N2, activation=f),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(N3, activation=f),
              tf.keras.layers.Dropout(dropout_rate),
              tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
            ])

    model.reset_states()

    model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

    model.fit(train_n, encoded_train_labels, epochs = epochs, verbose = 0)
    (loss_f[t], acc_f[t]) = model.evaluate(val_n, encoded_val_labels, verbose = 0)
    print(function_name[t] "=>", acc_f[t], "% accuracy")
    t++